# Agents Unplugged - for Real, no magic, no fuss
## LangChain, LangFlow and LangGraph

### Notebook overview (high‑level)


The diagram below shows the major sections and flows demonstrated in this notebook.



<details>


  <summary>Show/Hide overview diagram</summary>



```mermaid
graph TD
  A["Setup & Config"] --> B["LLM + Tools<BR>(search, calculator)"]
  B --> C["ReAct Agent demo"]
  C --> D["Micro‑evaluator"]
  B --> E["Optional:<BR>LangFlow API call"]
  B --> F["LangGraph:<BR>simple retrieve → answer"]
  F --> G["LangGraph:<BR> routed (calc vs retrieve)"]
  G --> H["Human‑in‑the‑loop graph"]
  H -->|interrupt_before human_review| I[(Paused state)]
  I -->|reject| R["Loop: retrieve → answer"]
  R --> H
  I -->|accept| J["Finalize to END"]
  J --> K["Helper: resume_to_end"]
  K --> L[(END state)]


  %% MCP: tools surfaced via a simple server
  B -.->|"also exposed via MCP"| S["Simple MCP Server<BR>Tools: search_policies, calculator"]
  subgraph MCP["MCP integration"]
    X["MCP Client / Bridge"]
  end
  C -->|"tool calls"| X
  F -->|"tool calls"| X
  G -->|"tool calls"| X
  H -->|"tool calls"| X
  X -->|"MCP protocol"| S


  %% optional visualization step
  J --> M["Optional: stream finalization (no interrupt)"]


  classDef section fill:#eef,stroke:#333,color:#000
  class A,B,C,D,E,F,G,H,I,J,K,L,M,S,X section
```


</details>




In [1]:
import json, os
file_name = 'config.json'  #paste path here
with open(file_name, 'r') as file:
    config = json.load(file)
    API_KEY = config.get("API_KEY") # Loading the API Key
    OPENAI_API_BASE = config.get("OPENAI_API_BASE") # Loading the API Base Url
    LANGFLOW_API_KEY = config.get("LANGFLOW_API_KEY") # Loading the Langflow API Key

In [2]:
# Importing the GPT-4 LLM and setting it up
os.environ['OPENAI_API_KEY']= API_KEY
os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE
os.environ["LANGFLOW_API_KEY"] = LANGFLOW_API_KEY

In [6]:
# Backend config — set ONE line
BACKEND = "OPENAI" # "OLLAMA"  # or "OPENAI"
MODEL_OPENAI = "gpt-4o-mini"
MODEL_OLLAMA = "llama3.1:8b"  # ensure it's pulled if using Ollama

import os, json, re, math, requests, ast, operator
from typing import List, Dict, Any

from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

try:
    if BACKEND == "OPENAI":
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model=MODEL_OPENAI, temperature=0.2)
    else:
        try:
            from langchain_community.chat_models import ChatOllama
        except Exception:
            from langchain_community.llms import Ollama as ChatOllama
        llm = ChatOllama(model=MODEL_OLLAMA)
except Exception as e:
    raise RuntimeError(f"Could not initialize LLM for BACKEND={BACKEND}: {e}")

llm.invoke("How are you doing today?")  # test if LLM is working

/home/asurace/github/Agents unplugged_ real_no_magic_no_fuss/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 13, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CL5xB2G5MR5Dhp3aZuzM20Gq8nprO', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8ccd08b8-3004-4673-abe1-4e97ee517c79-0', usage_metadata={'input_tokens': 13, 'output_tokens': 28, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [92]:
# Tiny policy corpus + tools
POLICIES = [
    {"id": "TravelPlus-2024", "text": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."},
    {"id": "DeviceCare-Pro", "text": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year."},
    {"id": "FleetAssist", "text": "FleetAssist covers rented vehicles during company travel. It excludes personal property inside the vehicle unless explicitly endorsed."},
]

def simple_search(query: str, top_k: int = 3):
    terms = [t.lower() for t in re.findall(r"\w+", query)]
    scored = []
    for doc in POLICIES:
        text = doc["text"].lower()
        score = sum(text.count(t) for t in set(terms))
        if score > 0:
            scored.append((score, doc))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [doc for _, doc in scored[:top_k]]

def search_policies(query: str) -> str:
    hits = simple_search(query, top_k=3)
    return json.dumps({"results": [{"id": h["id"], "snippet": h["text"][:280]} for h in hits]})

# Safe calculator tool using ast (updated for Python 3.12+)
OPS = {ast.Add: operator.add, ast.Sub: operator.sub, ast.Mult: operator.mul, ast.Div: operator.truediv,
       ast.Pow: operator.pow, ast.USub: operator.neg, ast.Mod: operator.mod, ast.FloorDiv: operator.floordiv}

def _eval(node):
    # Python 3.12+: ast.Constant replaces ast.Num
    if isinstance(node, ast.Constant) and isinstance(node.value, (int, float)):
        return node.value
    # Back-compat for < 3.8 code paths (harmless on >=3.8, will just be False)
    if hasattr(ast, "Num") and isinstance(node, ast.Num):
        return node.n
    if isinstance(node, ast.BinOp):
        return OPS[type(node.op)](_eval(node.left), _eval(node.right))
    if isinstance(node, ast.UnaryOp):
        return OPS[type(node.op)](_eval(node.operand))
    raise ValueError("Unsupported expression")

def calculator(expression: str) -> str:
    try:
        node = ast.parse(expression, mode='eval').body
        return str(_eval(node))
    except Exception as e:
        return f"Error: {e}"

tools = [
    Tool(name="search_policies", func=search_policies, description="Search a tiny policy corpus. Returns JSON."),
    Tool(name="calculator", func=calculator, description="Evaluate arithmetic like '2*(1500-200)'. Returns a number as text.")
]

# ReAct agent (ZERO_SHOT_REACT_DESCRIPTION) 

determines the goal directly from the questions without additional context. Here's how it works:

The agent understands the questions using:

Built-in prompt template: The ZERO_SHOT_REACT_DESCRIPTION agent uses a default prompt that instructs the LLM to:

Analyze the question
Reason about what information is needed
Use available tools to find answers
Formulate a response
Tool knowledge: The agent has access to tools (defined in the tools parameter) - search and calculator.

ReAct framework: The agent follows a "Reasoning + Acting" pattern where it:

Thinks about what the question is asking
Determines which tool to use
Uses the tool to get information
Continues reasoning until it reaches an answer
The agent doesn't need explicit contextual guidance because the combination of the question itself, the available tools, and the LLM's capabilities allow it to formulate a plan to answer each insurance-related query.

In [81]:
# Build the agent (ReAct)
agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True, handle_parsing_errors=True)

demo_questions = [
    "My work smartphone was stolen on a business trip. Is it covered, and what are the conditions?",
    "If a laptop worth CHF 1,800 is accidentally damaged twice in a year under DeviceCare-Pro, what's the max total payout after any deductibles?",
    "Does FleetAssist cover personal belongings inside a rental car?"
]
print("Demo questions:", *demo_questions, sep="\n- ")


Demo questions:
- My work smartphone was stolen on a business trip. Is it covered, and what are the conditions?
- If a laptop worth CHF 1,800 is accidentally damaged twice in a year under DeviceCare-Pro, what's the max total payout after any deductibles?
- Does FleetAssist cover personal belongings inside a rental car?


In [9]:
agent.invoke(demo_questions[0])



> Entering new AgentExecutor chain...
I need to find out if the theft of a work smartphone during a business trip is covered under any relevant policies and what the conditions might be. I'll search the policy corpus for information related to coverage for stolen work devices during business trips.  
Action: search_policies  
Action Input: "stolen work smartphone business trip coverage"  
Observation: {"results": [{"id": "TravelPlus-2024", "snippet": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."}, {"id": "DeviceCare-Pro", "snippet": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year."}]}
Though

{'input': 'My work smartphone was stolen on a business trip. Is it covered, and what are the conditions?',
 'output': 'Yes, your work smartphone is covered if it was stolen during a business trip. Under the TravelPlus-2024 policy, there is a deductible of CHF 200 for theft. The DeviceCare-Pro policy requires a police report to be filed within 48 hours of the theft, with a maximum payout of CHF 1,500 per device and a limit of 2 incidents per policy year.'}

In [10]:
agent.invoke(demo_questions[1])



> Entering new AgentExecutor chain...
To determine the maximum total payout after any deductibles for a laptop worth CHF 1,800 that is accidentally damaged twice in a year under DeviceCare-Pro, I need to find the relevant policy details regarding coverage limits and deductibles for accidental damage. 

Action: search_policies
Action Input: "DeviceCare-Pro accidental damage coverage limits and deductibles"

Observation: {"results": [{"id": "DeviceCare-Pro", "snippet": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year."}, {"id": "TravelPlus-2024", "snippet": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not c

/tmp/ipykernel_113865/1964599603.py:28: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num): return node.n
/tmp/ipykernel_113865/1964599603.py:28: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  if isinstance(node, ast.Num): return node.n


The maximum total payout for the two incidents of accidental damage is CHF 3,000. However, since the policy covers only up to CHF 1,500 per incident, the total payout for both incidents is CHF 3,000. 

Final Answer: CHF 3,000

> Finished chain.


{'input': "If a laptop worth CHF 1,800 is accidentally damaged twice in a year under DeviceCare-Pro, what's the max total payout after any deductibles?",
 'output': 'CHF 3,000'}

In [ ]:
agent.invoke(demo_questions[2])



> Entering new AgentExecutor chain...
I need to check if there are any policies related to FleetAssist that mention coverage for personal belongings inside a rental car. This will help me determine if such coverage exists. 
Action: search_policies
Action Input: "FleetAssist personal belongings rental car coverage"
Observation: {"results": [{"id": "TravelPlus-2024", "snippet": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."}, {"id": "FleetAssist", "snippet": "FleetAssist covers rented vehicles during company travel. It excludes personal property inside the vehicle unless explicitly endorsed."}, {"id": "DeviceCare-Pro", "snippet": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is fil

{'input': 'Does FleetAssist cover personal belongings inside a rental car?',
 'output': 'No, FleetAssist does not cover personal belongings inside a rental car unless explicitly endorsed.'}

# Let's evaluate the questions:

In [11]:
# Micro-evaluator
import pandas as pd
try:
    from caas_jupyter_tools import display_dataframe_to_user
except Exception:
    def display_dataframe_to_user(name, df):
        print(f"\n[name={name}]\n{df}")

def contains_all(text: str, phrases: list[str]) -> bool:
    low = text.lower()
    return all(p.lower() in low for p in phrases)

tests = [
    {"q": demo_questions[0], "must": ["covered", "theft", "police", "48"]},
    {"q": demo_questions[2], "must": ["not", "unless", "endorsed"]},
]
rows = []
for t in tests:
    out = agent.invoke(t["q"])
    ans = out["output"] if isinstance(out, dict) and "output" in out else str(out)
    rows.append({"question": t["q"], "passed": contains_all(ans, t["must"]), "answer": ans})

df = pd.DataFrame(rows)
display_dataframe_to_user("LangChain agent micro‑eval", df)
df




> Entering new AgentExecutor chain...
I need to find out if a stolen work smartphone is covered under a policy and what the specific conditions are for coverage. I'll search the policy corpus for relevant information regarding coverage for stolen work devices.  
Action: search_policies  
Action Input: "stolen work smartphone coverage"  
Observation: {"results": [{"id": "DeviceCare-Pro", "snippet": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year."}, {"id": "TravelPlus-2024", "snippet": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."}]}
Thought:I found two relevant policies regarding coverage fo

,question,passed,answer
0,My work smartphone was stolen on a business tr...,True,"Yes, your stolen work smartphone is covered un..."
1,Does FleetAssist cover personal belongings ins...,True,"No, FleetAssist does not cover personal belong..."


### Call your LangFlow flow (optional)

In [4]:
import requests, json
def run_langflow(flow_id: str, user_input: str, base_url: str = "http://127.0.0.1:7860/api/v1"):
    url = f"{base_url}/run/{flow_id}"
    payload = {"input_value": user_input, "output_type": "chat", "input_type": "chat"}
    headers = {
        "x-api-key": LANGFLOW_API_KEY  # Add your LangFlow API key here
    }
    r = requests.post(url, json=payload, headers=headers, timeout=60)
    r.raise_for_status()
    data = r.json()
    try:
        return data["outputs"][0]["outputs"][0]["results"]["message"]["text"]
    except Exception:
        return json.dumps(data, indent=2)

# Example:
print(run_langflow("55e5414b-966a-48dc-8bc4-bfb2e3168c72", "Hello from the notebook! Tell me what I can do with this integration between LangChain and LangFlow"))


Hello! It's fantastic to see your enthusiasm for exploring the integration between LangChain and LangFlow! This combination opens up a world of possibilities for building powerful applications that leverage language models and workflows. Here are some exciting things you can do with this integration:

1. **Streamlined Workflow Creation**: LangFlow provides a visual interface to design and manage workflows, making it easier to create complex applications without getting lost in code. You can drag and drop components to build your language processing pipelines.

2. **Custom Language Models**: With LangChain, you can easily integrate various language models, including OpenAI's GPT, Hugging Face models, or even your own fine-tuned models. This allows you to tailor the language processing capabilities to your specific needs.

3. **Data Processing Pipelines**: You can create workflows that preprocess data, perform language tasks (like summarization, translation, or sentiment analysis), and t

## LangGraph — stateful agent graphs
- **State schema** → **nodes** → **edges**
- Optional **checkpointer** (threaded memory)
- Below: a minimal retrieve→answer pipeline using our existing `llm` + retriever.


In [82]:
from typing import TypedDict
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()

# --- STATE MACHINE PART 1: THE STATE SCHEMA ---
# This TypedDict defines the structure of your state. 
# It's the memory that is passed between all nodes.
class AgentState(TypedDict, total=False):
    question: str
    context: str
    answer: str

# --- STATE MACHINE PART 2: THE NODES ---
# Each node is a function that operates on the state. It receives the current
# state and returns a dictionary to update it.
def retrieve_node(state: AgentState) -> AgentState:
    import json as _json
    raw = search_policies(state.get("question", ""))
    data = _json.loads(raw)
    snippets = [r.get("snippet","") for r in data.get("results", [])]
    return {"context": "\n\n".join(snippets) if snippets else "No policy hits."}

def answer_node(state: AgentState) -> AgentState:
    q = state.get("question",""); cx = state.get("context","")
    prompt = ("Use ONLY the policy snippets to answer. If insufficient, say so.\n\n"
              f"Policy snippets:\n{cx}\n\nQuestion: {q}\n\nAnswer:")
    out = llm.invoke(prompt)
    text = getattr(out, "content", str(out))
    return {"answer": text}

# --- STATE MACHINE PART 3: THE GRAPH DEFINITION (EDGES) ---
# Here you instantiate the state machine with your schema.
graph = StateGraph(AgentState)
graph.add_node("retrieve", retrieve_node)
graph.add_node("answer", answer_node)
graph.set_entry_point("retrieve")
# This edge defines a transition: after 'retrieve' always go to 'answer'.
graph.add_edge("retrieve", "answer")
# After 'answer', the machine stops.
graph.add_edge("answer", END)
app = graph.compile(checkpointer=checkpointer) if checkpointer else graph.compile()

print(app.invoke(
    {"question": "Does FleetAssist cover belongings in a rental car?"}, 
    config={"configurable": {"thread_id": "1"}}
).get("answer"))


No, FleetAssist does not cover personal property inside the vehicle unless explicitly endorsed.


In [60]:
# Optional: add a simple router for calc-like questions
import re

# --- STATE MACHINE: A CONDITIONAL EDGE ---
# This function inspects the state to decide which path to take.
# This is the "logic" of the state machine.
def route(state: AgentState) -> str:
    q = state.get("question","")
    return "calc" if re.search(r"\bCHF\b|\d+\s*[+\-*/]", q) else "retrieve"

def calc_node(state: AgentState) -> AgentState:
    q = state.get("question","")
    m = re.search(r"(\d+[\d\s+\-*/().]*)", q)
    expr = m.group(1) if m else None
    out = calculator(expr) if expr else "Error: no expression found"
    return {"context": f"Calculated: {expr} = {out}", "answer": str(out)}

g2 = StateGraph(AgentState)
g2.add_node("retrieve", retrieve_node)
g2.add_node("calc", calc_node)
g2.add_node("answer", answer_node)
# --- STATE MACHINE: DEFINING DYNAMIC TRANSITIONS ---
# This tells the machine to call the 'route' function at the START.
# Based on the string returned by 'route', it will transition to either
# the 'retrieve' node or the 'calc' node.
g2.add_conditional_edges(START, route, {"retrieve": "retrieve", "calc": "calc"})
g2.add_edge("retrieve", "answer")
g2.add_edge("calc", "answer")
g2.add_edge("answer", END)
app_routed = g2.compile()

print(app_routed.invoke({"question": "If a laptop worth CHF 1,800 is damaged twice, what's the total?"}).get("answer"))


/tmp/ipykernel_575846/1964599603.py:28: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num): return node.n
/tmp/ipykernel_575846/1964599603.py:28: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  if isinstance(node, ast.Num): return node.n


Insufficient information.


In [89]:
# Optional: add a human-in-the-loop for review
from typing import Literal

# --- STATE MACHINE: EXTENDING THE STATE SCHEMA ---
# We add 'human_feedback' to the state to handle the new logic.
class AgentStateWithHuman(AgentState):
    human_feedback: str | None

def human_review_node(state: AgentStateWithHuman) -> dict:
    # This node is a placeholder for the interruption.
    # The graph will pause here, and the user can provide feedback.
    return {}

# --- STATE MACHINE: A CYCLICAL CONDITIONAL EDGE ---
# This router function creates a loop. By inspecting the state for 'human_feedback',
# it can decide to either end the process or go back to the 'retrieve' node.
def route_feedback(state: AgentStateWithHuman) -> Literal["retrieve", "end"]:
    # Check for human feedback. If "reject", go back to retrieve. Otherwise, end.
    if state.get("human_feedback", "").lower() == "reject":
        return "retrieve"
    return "end"

g3 = StateGraph(AgentStateWithHuman)
g3.add_node("retrieve", retrieve_node)
g3.add_node("calc", calc_node)
g3.add_node("answer", answer_node)
g3.add_node("human_review", human_review_node)

g3.add_conditional_edges(START, route, {"retrieve": "retrieve", "calc": "calc"})
g3.add_edge("retrieve", "answer")
g3.add_edge("calc", "answer")
g3.add_edge("answer", "human_review")
# --- STATE MACHINE: DEFINING THE LOOP ---
# After the 'human_review' node, call 'route_feedback' to decide the next step.
# This is where the machine can either loop back to 'retrieve' or proceed to 'END'.
g3.add_conditional_edges("human_review", route_feedback, {"retrieve": "retrieve", "end": END})

# Compile the graph, interrupting before the human_review node
# app_human = g3.compile(interrupt_before=["human_review"])

# The 'interrupt_before' argument tells the state machine to pause at a specific point.
# The 'checkpointer' saves the state when it pauses.
app_human = g3.compile(checkpointer=checkpointer, interrupt_before=["human_review"])

# --- How to run this graph ---
# 1. Start the run
# initial_state = {"question": "Does FleetAssist cover belongings in a rental car?"}
# for step in app_human.stream(initial_state):
#     print(step)

# 2. The graph will pause at the 'human_review' step.
#    Inspect the answer, then continue the run with feedback.
# approved_state = app_human.get_state(step['__end__']) # Get the state of the paused run
# next_step = app_human.invoke(None, approved_state) # Continue with "approve"
# print(next_step)

# To reject:
# rejected_state = app_human.get_state(step['__end__'])
# rejected_state.values['human_feedback'] = 'reject'
# next_step = app_human.invoke(None, rejected_state) # Continue with "reject"
# print(next_step)

In [90]:
# initial_state = {"question": "Does FleetAssist cover belongings in a FLYING car?"}
# for step in app_human.stream(initial_state):
#     print(step)

import uuid
config = {"configurable": {"thread_id": str(uuid.uuid4())}}
initial_state = {"question": "Does FleetAssist cover belongings in a rentat car?"}
for step in app_human.stream(initial_state, config=config):
    print(step)

# After the stream has paused, you can inspect the current state
paused_state = app_human.get_state(config)
print("Current state values:", paused_state.values)

# To confirm it's paused, check the 'next' step in the checkpoint
if paused_state.next == ('human_review',):
    print("\nGraph is confirmed to be in a paused state, waiting to execute 'human_review'.")
else:
    print(f"\nGraph is not paused at the expected step. Next step: {paused_state.next}")

{'retrieve': {'context': 'TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered.\n\nDeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year.\n\nFleetAssist covers rented vehicles during company travel. It excludes personal property inside the vehicle unless explicitly endorsed.'}}
{'answer': {'answer': 'No, FleetAssist does not cover personal property inside the vehicle unless explicitly endorsed.'}}
{'__interrupt__': ()}
Current state values: {'question': 'Does FleetAssist cover belongings in a rentat car?', 'context': 'TravelPlus provides coverage for business travel including delayed flights, lost baggage, and

In [84]:
# Accept the answer and fully end the run
# next_step = app_human.invoke({}, config)
print("Accepting the answer and resuming. You should see 'retrieve' and 'answer' steps, then we'll advance past the human_review interrupt to END:")
for step in app_human.stream({"human_feedback": "accept"}, config):
    print(step)

# The stream above interrupts before 'human_review' again (by design).
# To actually execute 'human_review' and route to END, resume using a non-interrupting app compiled with the same checkpointer.
app_human_no_interrupt = g3.compile(checkpointer=checkpointer)
final_result = app_human_no_interrupt.invoke({}, config)
print("Final resume result:", final_result)

# Verify we've reached END: when finished, there should be no 'next' step.
current_state = app_human_no_interrupt.get_state(config)
if not getattr(current_state, "next", None):
    print("\nGraph reached END successfully.")
else:
    print(f"\nGraph is not at END. Next step: {current_state.next}")

Accepting the answer and resuming. You should see 'retrieve' and 'answer' steps, then we'll advance past the human_review interrupt to END:
{'retrieve': {'context': 'No policy hits.'}}
{'answer': {'answer': 'Insufficient information.'}}
{'__interrupt__': ()}
Final resume result: {'context': 'No policy hits.', 'answer': 'Insufficient information.', 'human_feedback': 'accept'}

Graph reached END successfully.


In [87]:
# Helper to resume a paused run and drive it to END

def resume_to_end(app_interrupting, graph, checkpointer, config, human_feedback: str | None = None):
    """
    Resume a graph that was compiled with interrupts and drive it to END using the same
    checkpointer + config (thread_id). If a human_feedback value is provided, it will be
    included in the resume payload to influence routing at 'human_review'.

    Returns a small summary dict with keys: result, at_end (bool), next (tuple|None), state (values).
    """
    # Compile a non-interrupting version of the same graph with the same checkpointer
    app_no_interrupt = graph.compile(checkpointer=checkpointer)

    # Build a payload only if we have a human_feedback update to apply
    payload = {"human_feedback": human_feedback} if human_feedback is not None else {}

    # Invoke once to proceed past any paused step (e.g., 'human_review') and continue to END
    result = app_no_interrupt.invoke(payload, config)

    # Inspect current state to confirm END
    state_snapshot = app_no_interrupt.get_state(config)
    next_step = getattr(state_snapshot, "next", None)
    at_end = not next_step

    return {
        "result": result,
        "at_end": at_end,
        "next": next_step,
        "state": state_snapshot.values,
    }

In [88]:
# Optional: stream finalization without interrupts (for visualization)
# Use this after you've accepted to see the final steps.
try:
    app_human_no_interrupt  # already compiled earlier?
except NameError:
    app_human_no_interrupt = g3.compile(checkpointer=checkpointer)

# Increase recursion limit to avoid premature stop on long traces
config_stream = dict(config)
config_stream["recursion_limit"] = 100

print("Streaming finalization steps (no interrupt):")
for step in app_human_no_interrupt.stream({}, config_stream):
    print(step)

state = app_human_no_interrupt.get_state(config_stream)
print("END reached?", "Yes" if not getattr(state, "next", None) else f"No, next={state.next}")

Streaming finalization steps (no interrupt):
{'retrieve': {'context': 'TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered.\n\nDeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year.\n\nFleetAssist covers rented vehicles during company travel. It excludes personal property inside the vehicle unless explicitly endorsed.'}}
{'answer': {'answer': 'No, FleetAssist does not cover personal property inside the rented vehicle unless explicitly endorsed.'}}
{'human_review': None}
END reached? Yes


In [91]:
# Reject and continue (do NOT end)
rejection_payload = {"human_feedback": "reject"}
print("Rejecting and re-running the graph. You should see 'retrieve' and 'answer' steps below, then a pause at 'human_review':")
for step in app_human.stream(rejection_payload, config):
    print(step)

# Confirm it's paused at 'human_review' again
current_state_after_rejection = app_human.get_state(config)
if current_state_after_rejection.next == ('human_review',):
    print("\nGraph is confirmed to be in a paused state, waiting to execute 'human_review'.")
else:
    print(f"\nGraph is not paused at the expected step. Next step: {current_state_after_rejection.next}")

# NOTE: The run remains paused. You can:
# - Provide new feedback (e.g., reject again) to further iterate, OR
# - Switch to acceptance later to finish, e.g.: resume_to_end(app_human, g3, checkpointer, config, human_feedback='accept')

Rejecting and re-running the graph. You should see 'retrieve' and 'answer' steps below, then a pause at 'human_review':
{'retrieve': {'context': 'TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered.\n\nDeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year.\n\nFleetAssist covers rented vehicles during company travel. It excludes personal property inside the vehicle unless explicitly endorsed.'}}
{'answer': {'answer': 'No, FleetAssist does not cover personal property inside the rented vehicle unless explicitly endorsed.'}}
{'__interrupt__': ()}

Graph is confirmed to be in a paused state, waiting to execute 'hu

### Diagram of the Human‑in‑the‑loop Routed Agent

This diagram reflects the latest graph (g3) with routing (calc vs retrieve), answering, a human_review pause, and feedback-based looping.

```mermaid
graph LR
    A[START] --> B{route question}
    B -->|Contains math/CHF| C[calc_node]
    B -->|Otherwise| D[retrieve_node]
    C --> E[answer_node]
    D --> E
    E --> H[human_review]
    H --> I{route_feedback}
    I -->|reject| D
    I -->|accept| F[END]

    %% Styling (optional)
    classDef calc fill:#f9f,stroke:#333,stroke-width:2px,color:#000
    classDef retv fill:#ccf,stroke:#333,stroke-width:2px,color:#000
    classDef ans  fill:#cfc,stroke:#333,stroke-width:2px,color:#000
    classDef human fill:#ffc,stroke:#333,stroke-width:2px,color:#000

    class C calc
    class D retv
    class E ans
    class H human
```

Notes:
- The app is compiled with `interrupt_before=["human_review"]`, so runs pause at `human_review`.
- On `reject`, `route_feedback` loops back to `retrieve_node` (you’ll see `retrieve` → `answer` → pause again).
- On `accept`, the run proceeds to `END` (use the non‑interrupting compiled app or the helper to finalize).


## Closing
Agents are **tools-first** systems. Ship‑ready means:
- Versioned graphs
- Safe MCP/allow‑listed tools
- Eval + observability

**Thanks!**